Email Spam detection

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Data loading and cleaning

In [48]:
df=pd.read_csv("spam_data.csv")
df.rename(columns={"label": "target"}, inplace=True)
df['target'] = df['target'].replace({'ham': 0, 'spam': 1})

df.head()



C:\Users\Acer\AppData\Local\Temp\ipykernel_51400\493497115.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['target'] = df['target'].replace({'ham': 0, 'spam': 1})


,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
